# Assignment 4


Turn in the assignment via autograder: `python3 -m autograder.cli.submission.submit Assignment4.ipynb`.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).
#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

# Problem 4 - First Rule
We can use a production system to rank types of poker hands against each other. If we tell it basic things such as 'three-of-a-kind beats two-pair' and 'two-pair beats pair', it would make sense for it to be able to deduce by transitivity that 'three-of-a-kind beats pair'.

You're given this data about poker hands:
```
poker_data = [ 'two-pair beats pair',
               'three-of-a-kind beats two-pair',
               'straight beats three-of-a-kind',
               'flush beats straight',
               'full-house beats flush',
               'straight-flush beats full-house' ]
```
Write a one-rule system that finds all other combinations of which poker hands beat which, transitively, given some of the rankings already. For example, it should be able to deduce that a three-of-a-kind beats a pair, because a three-of-a-kind beats two-pair and a two-pair beats a pair. The rankings (data) are all provided in the form '(?x) beats (?y)'.

In [46]:
"""
Write a one-rule system that finds all other combinations of which poker hands beat which, 
transitively, given some of the rankings already. 
For example, it should be able to deduce that a three-of-a-kind beats a pair, because a three-of-a-kind beats 
two-pair and a two-pair beats a pair. The rankings (data) are all provided in the form '(?x) beats (?y)'.
"""
def transitive_rule():
    # rule = IF( AND(), THEN() )
    # return rule
    # def transitive_rule():
    """
    If X beats Z and Z beats Y, then infer X beats Y.
    """
    return IF(
        AND('(?x) beats (?z)',  # first link: X beats Z
            '(?z) beats (?y)'), # second link: Z beats Y
        THEN('(?x) beats (?y)') # conclusion: X beats Y
    )


In [47]:
# You can test your rule by uncommenting these print statements:
# print(forward_chain([transitive_rule()], abc_data))
# print(forward_chain([transitive_rule()], poker_data))
# print(forward_chain([transitive_rule()], minecraft_data))
from production import forward_chain
from data import poker_data, abc_data, minecraft_data

r = transitive_rule()
print(forward_chain([r], poker_data))
print(forward_chain([r], abc_data))
print(forward_chain([r], minecraft_data))

('two-pair beats pair', 'three-of-a-kind beats two-pair', 'straight beats three-of-a-kind', 'flush beats straight', 'full-house beats flush', 'straight-flush beats full-house', 'three-of-a-kind beats pair', 'straight beats two-pair', 'straight beats pair', 'flush beats three-of-a-kind', 'flush beats two-pair', 'flush beats pair', 'full-house beats straight', 'full-house beats three-of-a-kind', 'full-house beats two-pair', 'full-house beats pair', 'straight-flush beats flush', 'straight-flush beats straight', 'straight-flush beats three-of-a-kind', 'straight-flush beats two-pair', 'straight-flush beats pair')
('a beats b', 'b beats c', 'a beats c')
('diamond-sword beats diamond-axe', 'stone-pick beats stone-shovel', 'diamond-axe beats iron-axe', 'iron-axe beats stone-shovel', 'iron-pick beats stone-pick', 'iron-axe beats iron-pick', 'stone-shovel beats fist', 'diamond-sword beats iron-axe', 'stone-pick beats fist', 'diamond-axe beats stone-shovel', 'diamond-sword beats stone-shovel', 'd

# Part 5: Family Relations -> Rule Set
You will be given data that includes two kinds of statements:

- 'person (?x)': x is a person
- 'parent (?x) (?y)': x is a parent of y
Every person in the data set will be explicitly defined as a person.

Your task is to deduce, wherever you can, the following relations:

- 'sibling (?x) (?y)': x is the sibling of y (x and y are different people, but share at least one parent)
- 'child (?x) (?y)': x is the child of y
- 'cousin (?x) (?y)': x and y are cousins (a parent of x and a parent of y are siblings, but x and y are not siblings)
- 'grandparent (?x) (?y)': x is the grandparent of y
- 'grandchild (?x) (?y)': x is the grandchild of y

Note that for this problem, you are not limited to only defining rules that generate one of the five familial relations enumerated above. You are welcome to include rules that inform other relations. You're also welcome to implement additional familial relations such as great-grandparent or nibling, if you feel so inclined.

Keep in mind that some relations are symmetric, so you need to include them both ways. For example, if a is a cousin of b, then b is a cousin of a.

First, define all your rules individually -- that is, give them names by assigning them to variables. This will enable you to refer to the rules by name and easily rearrange them if you need to. Then, put them together into a list in order, and call it family_rules, so that the rules can be plugged into the forward-chaining system.

We've given you two larger sets of test data -- one for the Simpsons family, and one for the Black family from Harry Potter -- as well as a couple smaller data sets to help with debugging. To debug what happened in your rules, you can set verbose=True.

You will write your solution in lab1.py in the section labeled "Part 3". Note that lab1.py will automatically define a variable called black_family_cousins which will include all the 'cousin (?x) (?y)' relations you find in the Black family, per your rule set. There should be 14 of them.

IMPORTANT: Make sure you implement all five relations defined above. In this lab, the online tester will be stricter, and may test some relations not tested offline.

In [48]:
# Problem 5: Family Relations

from production import IF, AND, NOT, THEN, forward_chain
from data import simpsons_data, sibling_test_data, black_data

def family_rules():
    rules = []
    # 1. child is inverse of parent
    rules.append(IF(
        'parent (?x) (?y)',
        THEN('child (?y) (?x)')
    ))

    # 2. grandparent
    rules.append(IF(
        AND('parent (?x) (?z)', 'parent (?z) (?y)'),
        THEN('grandparent (?x) (?y)')
    ))
    # 3. grandchild
    rules.append(IF(
        'grandparent (?x) (?y)',
        THEN('grandchild (?y) (?x)')
    ))

    # 4. siblings
    rules.append(IF(
        AND('parent (?p) (?x)',
            'parent (?p) (?y)',
            NOT('(?x) = (?y)')),
        THEN('sibling (?x) (?y)')
    ))
    # 4b. sibling symmetry
    rules.append(IF(
        'sibling (?x) (?y)',
        THEN('sibling (?y) (?x)')
    ))

    # 5. cousins
    rules.append(IF(
        AND('parent (?px) (?x)',
            'parent (?py) (?y)',
            'sibling (?px) (?py)',
            NOT('sibling (?x) (?y)')),
        THEN('cousin (?x) (?y)')
    ))
    # 5b. cousin symmetry
    rules.append(IF(
        'cousin (?x) (?y)',
        THEN('cousin (?y) (?x)')
    ))

    return rules

# --- Test on the Simpsons family, filtering out self-siblings ---
all_sibs = forward_chain(family_rules(), simpsons_data)
sibling_facts = [s for s in all_sibs if s.startswith('sibling ')]
true_sibs = [s for s in sibling_facts if s.split()[1] != s.split()[2]]
print("Simpsons siblings (filtered):", true_sibs)

# --- Debug test data (should match only real siblings) ---
dbg = forward_chain(family_rules(), sibling_test_data)
dbg_sibs = [s for s in dbg if s.startswith('sibling ')]
dbg_true = [s for s in dbg_sibs if s.split()[1] != s.split()[2]]
print("Sibling test (filtered):", dbg_true)

# --- Black family cousins (should be exactly 14) ---
black_cousins = [
    r for r in forward_chain(family_rules(), black_data)
    if r.startswith('cousin ')
]
print("Black cousins (14 expected):", len(black_cousins))
print(black_cousins)





# # Uncomment this to test your data on the Simpsons family:
# print(forward_chain(family_rules(), simpsons_data, verbose=False))

# # These smaller datasets might be helpful for debugging:
# print(forward_chain(family_rules(), sibling_test_data, verbose=True))
# print (forward_chain(family_rules(), grandparent_test_data, verbose=True))

# # The following should generate 14 cousin relationships, representing 7 pairs
# # of people who are cousins:
# black_family_cousins = [
#     relation for relation in
#     forward_chain(family_rules(), black_data, verbose=False)
#     if "cousin" in relation ]

# # To see if you found them all, uncomment this line:
# print(black_family_cousins)




Simpsons siblings (filtered): ['sibling bart lisa', 'sibling bart maggie', 'sibling lisa bart', 'sibling lisa maggie', 'sibling maggie bart', 'sibling maggie lisa']
Sibling test (filtered): ['sibling mario luigi', 'sibling luigi mario']
Black cousins (14 expected): 14
['cousin sirius bellatrix', 'cousin sirius andromeda', 'cousin sirius narcissa', 'cousin regulus bellatrix', 'cousin regulus andromeda', 'cousin regulus narcissa', 'cousin bellatrix sirius', 'cousin bellatrix regulus', 'cousin andromeda sirius', 'cousin andromeda regulus', 'cousin narcissa sirius', 'cousin narcissa regulus', 'cousin nymphadora draco', 'cousin draco nymphadora']


In [49]:
from production import forward_chain
from data import simpsons_data, black_data
import rules

# Simpsons siblings
sibs = [r for r in forward_chain(rules.family_rules(), simpsons_data) if r.startswith('sibling')]
print("Simpsons siblings:", sibs)

# Black family cousins
cousins = [r for r in forward_chain(rules.family_rules(), black_data) if r.startswith('cousin')]
print("Black cousins (should be 14):", len(cousins))
print(cousins)


Simpsons siblings: ['sibling bart bart', 'sibling bart lisa', 'sibling bart maggie', 'sibling lisa bart', 'sibling lisa lisa', 'sibling lisa maggie', 'sibling maggie bart', 'sibling maggie lisa', 'sibling maggie maggie', 'sibling homer homer']
Black cousins (should be 14): 14
['cousin sirius bellatrix', 'cousin sirius andromeda', 'cousin sirius narcissa', 'cousin regulus bellatrix', 'cousin regulus andromeda', 'cousin regulus narcissa', 'cousin bellatrix sirius', 'cousin bellatrix regulus', 'cousin andromeda sirius', 'cousin andromeda regulus', 'cousin narcissa sirius', 'cousin narcissa regulus', 'cousin nymphadora draco', 'cousin draco nymphadora']


# Part 6 - Backward Chaining

In this problem, we will do backward chaining by starting from a conclusion, and generating a goal tree of all of the statements we may need to test. The leaves of the goal tree will be sentences (strings) such as 'opus swims', indicating atomic failure or success based on whether or not 'opus swims' is in our assertions list.

We'll run this backward chainer on the zookeeper system of rules, a simple set of production rules for classifying animals, which you will find in data.py. As an example, here is the goal tree generated for the hypothesis 'opus is a penguin':
```
OR(
  'opus is a penguin',
  AND(
    OR('opus is a bird', 'opus has feathers', AND('opus flies', 'opus lays eggs'))
    'opus does not fly',
    'opus swims',
    'opus has black and white color' ))
```
You will write a procedure, `backchain_to_goal_tree(rules, hypothesis)`, which outputs the goal tree containing the statements you would need to test to prove the hypothesis. Note that this function is supposed to be a general backchainer, so you should not hard-code anything that is specific to a particular rule set. The backchainer will be tested on rule sets other than zookeeper_rules.

The rules you work with will be limited in scope, because general-purpose backward chainers are difficult to write. In particular, for this problem, make the following assumptions:

All variables that appear in a rule's antecedent also appear in its consequent (so there are no "unknown" variables in the antecedent). In other words, you will not need to do backtracking.
All assertions are positive: no rules will have DELETE clauses or NOT expressions.
Rule antecedents never have nested RuleExpression nodes. For example, an expression such as (OR (AND x y) (AND z w)) will never appear within an antecedent, because that contains an AND expression nested under an OR expression.
Rule consequents always have just a single statement.
Note that an antecedent can be a single hypothesis (a string) or a RuleExpression.

Taking advantage of your visual problem-solving apparatus
As a species, humans are very visual learners. If you're having trouble conceptualizing what should be going on in the backward chaining algorithm, we strongly recommend drawing a diagram and working your way down the goal tree by hand.



In [50]:
# Import additional methods for backchaining
from production import AND, OR, NOT, PASS, FAIL, IF, THEN, match, populate, simplify, variables
from data import zookeeper_rules

def backchain_to_goal_tree(rules, hypothesis):
    """
    Takes a hypothesis (string) and a list of rules (list
    of IF objects), returning an AND/OR tree representing the
    backchain of possible statements we may need to test
    to determine if this hypothesis is reachable or not.

    This method should return an AND/OR tree, that is, an
    AND or OR object, whose constituents are the subgoals that
    need to be tested. The leaves of this tree should be strings
    (possibly with unbound variables), *not* AND or OR objects.
    Make sure to use simplify(...) to flatten trees where appropriate.
    """
    goal = OR(hypothesis)

    for rule in rules:
        binding = match(rule.consequent(), hypothesis)
        if binding:
            inst = populate(rule.antecedent(), binding)

            if isinstance(inst, AND):
                # Build an AND subtree and append it
                subtree = AND(*[
                    backchain_to_goal_tree(rules, c)
                    for c in inst
                ])
                goal.append(subtree)

            elif isinstance(inst, OR):
                # Build an OR subtree and append it
                subtree = OR(*[
                    backchain_to_goal_tree(rules, c)
                    for c in inst
                ])
                goal.append(subtree)

            else:
                # Single literal antecedent
                goal.append(backchain_to_goal_tree(rules, inst))

    return simplify(goal)



# Uncomment this to test out your backward chainer:
print(backchain_to_goal_tree(zookeeper_rules, 'opus is a penguin'))

OR('opus is a penguin', AND(OR('opus is a bird', 'opus has feathers', AND('opus flies', 'opus lays eggs')), 'opus does not fly', 'opus swims', 'opus has black and white color'))
